In [1]:
import numpy as np
import os
from tqdm import tqdm
from scipy.stats import pearsonr, spearmanr
import scipy.sparse as sparse
from scipy.stats import bernoulli, poisson
import analysis_utils_mine as utils

import plotly.graph_objects as go
import chart_studio
import json
import pandas as pd
import ast
from datetime import datetime
import torch
import pandas as pd
from datetime import datetime, timedelta
import pickle

import matplotlib.pyplot as plt

In [2]:
project_dir = os.path.abspath('../data/floor_speeches_congs_115_116/') 
fit_dir = os.path.join(project_dir, "mallet_fits_removed_procedural_speeches")

# Load TBIP data.
data_dir = os.path.join(project_dir, "clean_removing_procedural")
(counts, vocabulary, author_indices, 
 author_map) = utils.load_text_data(data_dir)

# Load TBIP parameters.
param_dir = os.path.join(project_dir, "tbip-pytorch-fits-og-rem-procedural-speeches-k50-init-mallet/params/")
(document_loc, document_scale, objective_topic_loc, objective_topic_scale, 
 ideological_topic_loc, ideological_topic_scale, ideal_point_loc, 
 ideal_point_scale) = utils.load_tbip_parameters(param_dir)

# Compute means from variational parameters
document_mean = np.exp(document_loc + document_scale ** 2 / 2)
objective_topic_mean = np.exp(objective_topic_loc + 
                              objective_topic_scale ** 2 / 2)
ideological_topic_mean = ideological_topic_loc
ideal_point_mean = ideal_point_loc

In [3]:
print(document_mean.shape)
print(objective_topic_mean.shape)
print(ideological_topic_mean.shape)
print(ideal_point_mean.shape)
#ideal_point_mean = -1*ideal_point_mean
#ideological_topic_loc = ideological_topic_loc * -1

(58932, 50)
(50, 11433)
(50, 11433)
(490,)


In [58]:
utils.print_topics(objective_topic_loc, 
                   objective_topic_scale, 
                   ideological_topic_loc, 
                   ideological_topic_scale, 
                   vocabulary, 
                   20)

['Ideal Point = -1.0, Topic = 0: undermine, public, including, enforcement, opposition, require, law, safety, information, oppose, balance, required, protections, reasons, concerns, concerned, provide, process, federal, states'
 'Ideal Point = -0.5, Topic = 0: public, law, federal, balance, act, including, process, information, require, reserve, legislation, provide, required, current, ensure, concerns, requirements, consume, already, safety'
 'Ideal Point = 0.0, Topic = 0: federal, act, process, balance, reserve, law, legislation, public, government, current, information, provide, ensure, consume, requires, agencies, however, administration, require, requirements'
 'Ideal Point = 0.5, Topic = 0: federal, act, government, process, reserve, balance, colleagues, legislation, administration, law, current, regulations, federal government, agencies, agency, regulatory, ensure, however, simply, requires'
 'Ideal Point = 1.0, Topic = 0: federal, government, colleagues, act, bureaucrats, feder

In [16]:
utils.print_topics(objective_topic_loc, 
                   objective_topic_scale, 
                   ideological_topic_loc, 
                   ideological_topic_scale, 
                   vocabulary, 
                   50)

['Ideal Point = -1.0, Topic = 0: undermine, public, including, enforcement, opposition, require, law, safety, information, oppose, balance, required, protections, reasons, concerns, concerned, provide, process, federal, states, serious, critical, already, reserve, act, based, legislation, requirements, ability, ensure, effect, existing, standard, current, provision, evidence, changes, prohibit, consume, likely, certain, strongly, letter, example, persons, dangerous, national, requires, difficult, standards'
 'Ideal Point = -0.5, Topic = 0: public, law, federal, balance, act, including, process, information, require, reserve, legislation, provide, required, current, ensure, concerns, requirements, consume, already, safety, opposition, requires, ability, however, states, based, changes, oppose, existing, undermine, certain, agencies, provision, currently, protections, agency, rules, standards, policy, example, important, enforcement, necessary, clear, whether, access, administration, rea

In [6]:
%%capture cap --no-stderr
utils.print_topics(objective_topic_loc, 
                   objective_topic_scale, 
                   ideological_topic_loc, 
                   ideological_topic_scale, 
                   vocabulary, 
                   20)
with open('../../speeches_results/topics.txt', 'w') as f:
    f.write(cap.stdout)

In [7]:
legis_info = json.load(open(os.path.join(project_dir, 'legislator-info-1990-2020.json')))
print(len(legis_info))

1880


In [8]:
legis_id_to_info = {}
for x in legis_info:
    legis_id_to_info[x['id']['bioguide']] = x

In [9]:
parties = []
for a in author_map:
    x = legis_id_to_info[a]
    parties.append(x['terms'][-1]['party'])
print(len(parties))

490


In [10]:
names = []
for a in author_map:
    x = legis_id_to_info[a]
    names.append(x['id']['wikipedia'])
print(len(names))

490


In [11]:
colors = np.array(["steelblue" if p=='Democrat' else "crimson" for p in parties])
def get_ideological_topics(objective_topic_loc, 
                           objective_topic_scale,
                           ideological_topic_loc, 
                           ideological_topic_scale,
                           ideal_point):
    ideological_topic_mean = np.exp(objective_topic_loc +
                              ideal_point * ideological_topic_loc +
                              (objective_topic_scale ** 2 + 
                               ideal_point ** 2 * 
                               ideological_topic_scale ** 2) / 2)
    return ideological_topic_mean

In [12]:
ideal_points = ideal_point_mean
author_map = np.array(author_map)
print(author_map.shape)
ideal_points = np.array(ideal_points)
print(ideal_points.shape)

(490,)
(490,)


In [13]:
bid_to_tbip = {}
for i in range(len(author_map)):
    bid_to_tbip[author_map[i]] = ideal_points[i]
print(len(bid_to_tbip))

490


In [14]:
pickle.dump(bid_to_tbip, open('../../speeches_results/bid_to_tbip_floor_speeches.pkl', 'wb'))

In [19]:
name = "Liz Cheney"
print('Overall Idea Point of ' + name + ' = ' + str(bid_to_tbip[author_map[names.index(name)]]))


Overall Idea Point of Liz Cheney = -0.28457028


In [20]:
name = "Elise Stefanik"
print('Overall Idea Point of ' + name + ' = ' + str(bid_to_tbip[author_map[names.index(name)]]))


Overall Idea Point of Elise Stefanik = -0.16373397


In [21]:
import chart_studio
chart_studio.tools.set_credentials_file(username='pg96', api_key='NDcsdaKUnDc6rrurwLmg')
import chart_studio.plotly as py

In [22]:
def save_interactive(topic_number):
    topic_name = 'Topic ' + str(topic_number)

    fig = go.Figure(layout = go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'))
    x_min = -1.0
    x_max = 1.0
    diff = 0.05
    offset = -x_min / diff
    scatterplot_location = -0.53

    ideal_point_dict = {}
    for i in range(int((x_max - x_min) / diff) + 1):
        ideal_point_dict[i] = (i - offset) * diff

    # Add black line.
    fig.add_trace(go.Scatter(x=[x_min, x_max], 
                             y = [scatterplot_location, scatterplot_location],
                             line=dict(color="black", width=1),
                             marker=dict(size=1),
                             hoverinfo='skip'))

    # Add trace for all representatives
    fig.add_trace(go.Scatter(
        mode="markers",
        x=ideal_points, 
        y=scatterplot_location * np.ones(len(ideal_points)), 
        text=[x for x in names], 
        hoverinfo="x+text",
        marker=dict(color=colors, size=8)))

    num_top_words = 10

    # Add bar plots, one for each ideal point
    for step in np.arange(len(ideal_point_dict.keys())):
        ideological_topic_mean = get_ideological_topics(objective_topic_loc, 
                                   objective_topic_scale,
                                   ideological_topic_loc, 
                                   ideological_topic_scale,
                                   ideal_point_dict[step])
        topic_intensities = ideological_topic_mean[topic_number]
        top_topic_intensities = np.sort(topic_intensities)[-num_top_words:]
        top_topic_words = vocabulary[np.argsort(-topic_intensities)[:num_top_words]][::-1]
        fig.add_trace(
            go.Bar(
                visible=False,
                x=top_topic_intensities * 5,
                orientation='h',
                text=top_topic_words,
                textposition='outside',
                marker_color='rgb(175,122,197)',
                y0=0.529,
                dy=0.214,
                base=-0.3,
                hoverinfo='skip',
            ))

    fig.update_layout(go.Layout(
    yaxis = {'showgrid': False,
             'showline': False,
             'zeroline': False,
            'showticklabels': False,
            'fixedrange': True}),
    xaxis = {'showgrid': False,
             'showline': False,
             'zeroline': False,
            'showticklabels': False,
            'fixedrange': True})
    fig.update_yaxes(range=[-0.60, 2.5])
    fig.update_xaxes(range=[x_min - 0.01, x_max + 0.02])

    # Set default trace
    fig.data[len(fig.data) // 2].visible = True

    # Create and add slider
    steps = []
    for i in range(2, len(fig.data)):
        step = dict(
            method="update",
            args=[{"visible": [True, True] + [False] * (len(fig.data) - 2)}],  
            label="{:.2f}".format(ideal_point_dict[i - 2]),
        )
        step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active=(len(fig.data) // 2 - 1),
        currentvalue={"prefix": "Ideal Point: "},
        y=0.35,
        steps=steps)]

    fig.update_layout(sliders=sliders, showlegend=False, 
                      title={'text': "Word Usage as a Function of Ideal Point (Topic: {})".format(topic_name),
                             'x': 0.5,
                             'y': 0.9},
                      xaxis_title='Representative ideal points (hover to see names)',
                      annotations = [
                                    dict(xref='paper',
                                            yref='paper',
                                            x=0.5, y=0.255,
                                            font={'size': 14},
                                            showarrow=False,
                                            text ='Move slider to change ideal point')
                      ])
    #fig.show()
    fig.write_html("../../speeches_results/interactive_htmls/topic" + str(topic_ind) + "_interactive.html")

In [23]:
num_topics = objective_topic_mean.shape[0]
print(num_topics)

50


In [24]:
for topic_ind in range(num_topics):
    save_interactive(topic_ind)

In [25]:
print(document_mean.shape)

(58932, 50)


In [21]:
speeches_data = pd.read_csv(os.path.join(project_dir, "finalized_tbip_speech_set_raw_original_data_floor_speeches_house_after_removing_procedural_speeches.csv"))
print(speeches_data.info())
speeches_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58932 entries, 0 to 58931
Data columns (total 5 columns):
Speaker_Bioguide_ID    58932 non-null object
Speaker_Name           58932 non-null object
Text                   58932 non-null object
Date                   58932 non-null object
Legislative Body       58932 non-null object
dtypes: object(5)
memory usage: 2.2+ MB
None


,Speaker_Bioguide_ID,Speaker_Name,Text,Date,Legislative Body
0,M001201,Mr. MITCHELL,"Mr. Speaker, I rise today in the spirit of Mad...",2017-07-20,House
1,B001250,Mr. BISHOP of Utah,"Mr. Chair, I include in the Record my statemen...",2017-07-20,House
2,B001250,Mr. BISHOP of Utah,"Mr. Chair, I yield 5 minutes to the gentleman ...",2017-07-20,House
3,Y000033,Mr. YOUNG of Alaska,"Mr. Chairman, this is an issue that should hav...",2017-07-20,House
4,G000551,Mr. GRIJALVA,"Mr. Chairman, I yield myself such time as I ma...",2017-07-20,House


In [27]:
final_legis_info_and_tbip_data = pd.read_csv('../../legislator_info_and_tbip_congresses_115_and_116.csv')
# NOTE: while above file is ultimately created using the ideal point estimates... we only create these other files
# as resulting files for legislators ultimately retained in the final file containing everything used for research.

included_bid_to_tbip = dict(zip(final_legis_info_and_tbip_data.Bioguide_ID, 
                       final_legis_info_and_tbip_data.TBIP_Floor_Speeches))
# relev_bids = []
# for bid in included_bid_to_tbip:
#     if np.isnan(included_bid_to_tbip[bid]):
#         continue
#     relev_bids.append(bid)
# print(len(relev_bids))

In [28]:
#final_legis_info_and_tbip_data = pd.read_csv('legislator_info_and_tbip_congresses_115_and_116.csv')
print(final_legis_info_and_tbip_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 55 columns):
Bioguide_ID                                     506 non-null object
Name                                            506 non-null object
Gender                                          506 non-null object
Party                                           506 non-null object
Born                                            506 non-null int64
Number_of_House_Terms                           506 non-null int64
Present_Cong115                                 506 non-null int64
Present_Cong116                                 506 non-null int64
House_Election_Candidate_Vote_Share_2016        421 non-null float64
House_Election_Candidate_Vote_Share_2018        421 non-null float64
District                                        506 non-null object
District_Presidential_VoteShare_Dem2016         506 non-null float64
District_Presidential_VoteShare_GOP2016         506 non-null float64
District_De

In [29]:
speaker_ids_from_data = list(speeches_data['Speaker_Bioguide_ID'])
texts_from_data = list(speeches_data['Text'])
dates_from_data = list(speeches_data['Date'])

In [30]:
relev_bids = list(bid_to_tbip.keys())
print(len(relev_bids))

490


In [31]:
retain_inds = []
all_relev_ids, all_relev_texts, all_relev_dates = [], [], []
for i, bid in enumerate(speaker_ids_from_data):
    if bid in relev_bids:
        retain_inds.append(i)
        all_relev_ids.append(bid)
        all_relev_texts.append(texts_from_data[i])
        all_relev_dates.append(dates_from_data[i])
print(len(retain_inds))

58932


In [32]:
document_mean = document_mean[retain_inds]
document_mean = np.array(torch.softmax(torch.from_numpy(document_mean), 1))
print(document_mean.shape)

(58932, 50)


In [33]:
texts_topics = pd.DataFrame()
texts_topics["Bioguide_ID"] = all_relev_ids
texts_topics["Text"] = all_relev_texts
texts_topics["Date"] = all_relev_dates

In [34]:
for topic_ind in range(document_mean.shape[1]):
    texts_topics['Topic' + str(topic_ind)] = list(map(lambda x:np.float32(str(x)), list(document_mean[:, topic_ind])))
    

In [35]:
texts_topics.to_csv('../../speeches_results/texts_topic_proportions.csv', 
                    index=False)

In [36]:
relev_bid_to_inds = {}
for bid in relev_bids:
    relev_bid_to_inds[bid] = []
for i, idd in enumerate(all_relev_ids):
        relev_bid_to_inds[idd].append(i)

In [37]:
def get_mean_topic_props_author(X, bid, author_to_inds):
    return np.mean(X[author_to_inds[bid]], 0).reshape((1, num_topics))

In [38]:
relev_bid_avg_topic_props = []
for bid in relev_bids:
    relev_bid_avg_topic_props.append(get_mean_topic_props_author(document_mean, 
                                                                 bid,
                                                                 relev_bid_to_inds))
relev_bid_avg_topic_props = np.concatenate(relev_bid_avg_topic_props, 0)
print(relev_bid_avg_topic_props.shape)

(490, 50)


In [39]:
author_topic_props = pd.DataFrame()
author_topic_props["Bioguide_ID"] = relev_bids

In [40]:
for topic_ind in range(num_topics):
    author_topic_props['Topic' + str(topic_ind)] = list(relev_bid_avg_topic_props[:, topic_ind])
    

In [41]:
author_topic_props.to_csv('../../speeches_results/topic_proportions_per_author.csv',
                          index=False)